<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Collaborative filtering in recommender systems</div>

1. [Introduction](#sec1)
    1. [Recommender systems](#sec1-1)
    2. [MovieLens dataset](#sec1-2)
2. [Building a recommender system](#sec2)
    1. [Nearest Neighborhood algorithm](#sec2-1)
    2. [Similarity metrics](#sec2-2)
3. [Improving the model](#sec3)
    1. [Non-negative Matrix Factorization](#sec3-1)
    2. [A working recommender system](#sec3-2)

### Requirements

- pandas
- numpy
- timeit

In [1]:
import pandas as pd
import numpy as np
import warnings
import timeit
warnings.filterwarnings("ignore")

# <a id="sec1"></a> 1. Introduction

## <a id="sec1-1"></a> 1.1 Recommender systems

A recommendation system is an information filtering system that seeks to predict the "rating" or "preference" a user would give to an item. It is widely used in different online business such as Amazon, Netflix, Spotify, or social media like Facebook and Youtube. By using recommender systems, those companies are able to provide better or more suited products/services/contents that are personalized to a user based on his historical behavior.

Research on recommender algorithms garnered significant attention in 2006 when Netflix launched the Netflix Prize to improve the state of movie recommendation. The objective of this competition was to build a recommender algorithm that could beat their internal CineMatch algorithm in offline tests by 10%. It sparked an intense activity, both in academia and amongst hobbyists. The $1 M prize demonstrates the value that vendors place on accurate recommendations.

<img src="img/Homer.png" width="500px">
<i>Source: <a>https://medium.com/tiket-com-dev-team/build-recommendation-engine-using-graph-cbd6d8732e46
</a></i>
<br><br>

To build a recommender system, the most two popular approaches are Content-based and Collaborative Filtering.<br><br>
The Content-based approach will consist of learning the user's behavior from external information which are provided on the users (like their age, country of residence, ...) and on the items (like in the case of movies: genre, director, year, ...).<br><br>
Collaborative Filtering, on the other hand, doesn’t need anything else except users’ historical preference on a set of items. Because it’s based on historical data, the core assumption here is that the users who have agreed in the past tend to also agree in the future.
<br><br>
In the following document, we focus exclusively on this second approach.


## <a id="sec1-2"></a> 1.2 The MovieLens dataset
<br>
We choose to use a public dataset from the MovieLens web site (<a>http://movielens.org</a>) made available by GroupLens Research. These files contain 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users who joined MovieLens in 2000.

In [2]:
df_movies = pd.read_csv('data/ml-1m/movies.dat',sep='::',header=None)
df_movies.columns = ['movie_id', 'title', 'genre']
df_movies.movie_id = df_movies.movie_id - 1

df_users = pd.read_csv('data/ml-1m/users.dat',sep='::',header=None)
df_users.columns = ['user_id', 'gender', 'age','occupation','zipcode']
df_users.user_id = df_users.user_id - 1

df = pd.read_csv('data/ml-1m/ratings.dat',sep='::',header=None)
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df.movie_id = df.movie_id - 1
df.user_id = df.user_id - 1

The dataset consists of three files:
- The first contains information on users: gender, age, occupation and zipcode.
- The second contains information on movies: title, year and genre.
- The third contains the ratings, represented by an integer value from 1 to 5, with the timestamp of when it was made.

In [3]:
df_users.sample(10)

,user_id,gender,age,occupation,zipcode
2159,2159,F,18,19,46556
3678,3678,M,25,4,68108
3892,3892,M,25,6,79401
2419,2419,M,50,7,37938
1065,1065,M,45,12,02151
5694,5694,F,35,3,21221
3811,3811,M,35,17,78749
152,152,M,1,10,51537
4787,4787,M,35,7,85331
24,24,M,18,4,01609


In [4]:
df_movies.sample(10)

,movie_id,title,genre
874,885,Bulletproof (1996),Action
3362,3430,Death Wish II (1982),Action|Drama
46,46,Seven (Se7en) (1995),Crime|Thriller
1053,1066,"Damsel in Distress, A (1937)",Comedy|Musical|Romance
120,121,Boomerang (1992),Comedy|Romance
1780,1848,Prince Valiant (1997),Adventure
487,490,"Man Without a Face, The (1993)",Drama
2223,2291,Overnight Delivery (1996),Romance
1870,1938,"Best Years of Our Lives, The (1946)",Drama|War
2059,2127,Safe Men (1998),Comedy


In [5]:
df.sample(10)

,user_id,movie_id,rating,timestamp
801381,4801,3825,3,995719068
657209,3960,1982,3,967227560
549032,3388,647,3,967515695
13889,110,2027,5,977511594
649311,3912,1887,3,965766227
985497,5953,2142,1,957651428
560807,3449,3254,3,967244085
939931,5674,1502,2,958685254
95444,638,1196,2,981088461
655493,3947,1277,5,965682239


In [6]:
nb_users = max(df.user_id.unique()+1)
nb_movies = max(df.movie_id.unique()+1)
print("Distinct users:",nb_users)
print("Distinct movies:",nb_movies)

Distinct users: 6040
Distinct movies: 3952


In order to build a recommender system, we first have to convert the ratings' dataframe into a matrix in which each row represents all the ratings made by a user.
<br><br>
This step can take a couple of minutes.

In [7]:
rating_matrix = np.empty([nb_users,nb_movies])
rating_matrix.fill(np.nan)
def func(x):
    rating_matrix[x[0],x[1]] = x[2]

df.apply(func,axis = 1)

print(rating_matrix)

[[ 5. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [ 3. nan nan ... nan nan nan]]


In [8]:
rating_matrix.shape

(6040, 3952)

In [9]:
print("Percentage of NA in the matrix:",round((np.isnan(rating_matrix).sum()/rating_matrix.size)*100,2),"%")
for i in range(1,6):
    print("Percentage of",i,"in the matrix:",round(((rating_matrix==i).sum()/rating_matrix.size)*100,2),"%")

Percentage of NA in the matrix: 95.81 %
Percentage of 1 in the matrix: 0.24 %
Percentage of 2 in the matrix: 0.45 %
Percentage of 3 in the matrix: 1.09 %
Percentage of 4 in the matrix: 1.46 %
Percentage of 5 in the matrix: 0.95 %


The matrix is very sparse, because obviously most users have only watched and rated a minor part of all the referenced movies.

In [10]:
print("Average number of movies rated by user:",round((~np.isnan(rating_matrix)).sum(1).mean(),2))
print("Average rating:",round(np.nanmean(rating_matrix),2))
print("Minimum number of movies rated by a user:", (~np.isnan(rating_matrix)).sum(1).min())

Average number of movies rated by user: 165.6
Average rating: 3.58
Minimum number of movies rated by a user: 20


# <a id="sec2"></a> 2. Building a recommender system

## <a id="sec2-1"></a> 2.1 Nearest Neighborhood algorithm

The standard method of Collaborative Filtering is known as Nearest Neighborhood algorithm. There are two approaches: user-based or item-based. Here we will consider User-based Collaborative Filtering.
<br><br>
<img src="img/collaborative_filtering.png" width="500px">
<i>Source: <a>https://dzone.com/articles/recommendation-engine-models
</a></i>
<br><br>
We have an n × m matrix of ratings, with user uᵢ, i = 1, ...n and item pⱼ, j=1, …m. Now we want to predict the rating rᵢⱼ if target user i did not watch/rate an item j. The process is to calculate the similarities between target user i and all other users, select the top k similar users, and take the weighted average of ratings from these k users with similarities as weights.
<br><br>
$$r_{ij} = \frac{\sum_{k} similarity(u_i,u_k)r_{kj}}{\sum_{k} similarity(u_i,u_k)}$$
<br><br>
While different people may have different baselines when giving ratings, some people tend to give high scores generally, some are pretty strict even though they are satisfied with items. To avoid this bias, we can subtract each user’s average rating of all items when computing weighted average, and add it back for target user, shown as below.
<br><br>
$$r_{ij} = \bar{r_i} + \frac{\sum_{k} similarity(u_i,u_k)(r_{kj}-\bar{r_{k}})}{\sum_{k} |similarity(u_i,u_k)|}$$
<br><br>


## <a id="sec2-2"></a> 2.2 Similarity metrics

Each user is considered as a incomplete vector of which we only know a few components. Nevertheless we can compute a similarity value between a pair of users by only considering the ratings shared by both users.

There are multiple similarity functions that can be used to render the closeness between two vectors. <br><br>
Here we will consider two metrics:
- The Cosine similarity: $$Sim(u_i,u_k) = \frac{\sum_j r_{ij}r_{kj}}{\sqrt{\sum_j r_{ij}^2 \sum_j r_{kj}^2}}$$
- The Pearson Correlation: $$Sim(u_i,u_k) = \frac{\sum_j (r_{ij}-\bar{r_i})(r_{kj}-\bar{r_k})}{\sqrt{\sum_j (r_{ij}-\bar{r_i})^2 \sum_j (r_{kj}-\bar{r_k})^2}}$$

<br><br>
An implementation of these functions is given below.

In [11]:
# Reusable tool code

def pearson_similarity(u1,u2,target_item=None):
    
    # If an item is specified, remove it from the ratings vectors
    if target_item != None:
        u1 = np.delete(u1,target_item)
        u2 = np.delete(u2,target_item)
        
    # Identify the movies rated by both users
    common_ratings = np.logical_and(~np.isnan(u1), ~np.isnan(u2))

    u1 = u1[common_ratings]
    u2 = u2[common_ratings]
    u1 -= u1.mean()
    u2 -= u2.mean()
    
    res = ((u1**2).sum() * (u2**2).sum())
    
    # If no shared rating is found, the function returns 0
    if res==0:
        return 0
    return (u1*u2).sum() / (res)**(1/2)
    

def cosine_similarity(u1,u2,target_item=None):
    
    # If an item is specified, remove it from the ratings vectors
    if target_item != None:
        u1 = np.delete(u1,target_item)
        u2 = np.delete(u2,target_item)
        
    # Identify the movies rated by both users
    common_ratings = np.logical_and(~np.isnan(u1), ~np.isnan(u2))

    u1 = u1[common_ratings]
    u2 = u2[common_ratings]

    # If no shared rating is found, the function returns 0
    res = ((u1**2).sum() * (u2**2).sum())
    if res==0:
        return 0
    return (u1*u2).sum() / (res)**(1/2)

Given these similarity metrics, we can now predict user's ratings:

<div class="alert alert-warning">

**Exercice:**<br>
Implement the function predicting the rating of an item by a user based on the top k most similar users' ratings.
The parameters are as following:
- user is the user's vector of rating, corresponding to his row in the rating matrix
- train_set is the matrix of ratings used for comparison, thus not including the target user
- item is the index of the movie to be predicted
- similarity is the metric used (two values are allowed, 'pearson' or 'cosine')
- k is the number of similar users to consider

The function should return the rating directly as a float value.
</div>

In [13]:
# %load solutions/predict_rating.py
# Uncomment the line above to load a correction to this exercise.

# Reusable tool code

def predict_rating(user,train_set,item,similarity='pearson',k=100):
    
    mean_rating = np.nanmean(np.delete(user,item))
    
    similarities = []
    if similarity == 'pearson':
        similarities = [(u,pearson_similarity(user,train_set[u,:],target_item=item)) for u in range(train_set.shape[0])]
    if similarity == 'cosine':
        similarities = [(u,cosine_similarity(user,train_set[u,:],target_item=item)) for u in range(train_set.shape[0])]
        
    similarities = sorted(similarities, key=lambda x: x[1],reverse=True)[:k]

    sum_similarities = 0
    pred = 0
    for u,s in similarities:
        rating = train_set[u,item]
        if ~np.isnan(rating):
            pred += (rating - np.nanmean(train_set[u,:])) * s
            sum_similarities += s
    
    if sum_similarities != 0:
        pred /= sum_similarities
    
    return pred+mean_rating
    

<div class="alert alert-warning">
Try your function using the cell below and see how different your prediction is from the true rating.
</div>

In [14]:
print("Predicted rating:",predict_rating(rating_matrix[654,:],np.delete(rating_matrix,654,0),item=3947,k=20))
print("Real rating:",rating_matrix[654,3947])

Predicted rating: 2.660832140941065
Real rating: 2.0


The choice of the parameter k can sensibly affect the results: if k is too small then the most similar users picked will probably not have rated the target item, and if k is too big then all the less similar users considered will add noise to the result and will lower the model's precidion. The optimal value of k specific to the dataset, however Herlocker et al. found <b>k=20</b> to be a good starting value in general.

Our model turns out to be very unefficient, as we can see by computing the time needed to output a single prediction:

In [15]:
pred_time = timeit.timeit('predict_rating(rating_matrix[654,:],np.delete(rating_matrix,654,0),16,1000)', number=10, globals=globals())/10
print("Execution time:",pred_time,"s")

Execution time: 0.13860564709999607 s


There are two problems with our approach which explains this complexity:
- Comparing two users is expensive as they each have thousands of attributes,
- Finding top k similar users requires comparing the target user to every other.
<br><br>


In order to evaluate the model we can use the Mean Average Error (MAE), but various other metrics are used.<br><br>
$$MAE = \frac{1}{n_un_i}\sum_{u,i}e(u,i)$$
with the following error function: 
$$e(u,i) = |pred(u,i) - x_{ui}|$$

In this case, due to the amount of time needed to compute the error, we are not able to evaluate the whole system.
Nevertheless, we can still try and compare our two metrics on a single test user using the error function defined above:

In [16]:
# This cell takes a few minutes to run

test_user = 33
rated_movies = np.where(~np.isnan(rating_matrix[test_user,:]))[0]
train_set = np.delete(rating_matrix,test_user,0)

true_ratings = rating_matrix[test_user,rated_movies]
pred_ratings_pearson = [predict_rating(user=rating_matrix[test_user,:],item=m,similarity='pearson',train_set=train_set,k=100) for m in rated_movies]
pred_ratings_cosine = [predict_rating(user=rating_matrix[test_user,:],item=m,similarity='cosine',train_set=train_set,k=100) for m in rated_movies]

In [17]:
print("Error with Pearson similarity:",np.absolute(pred_ratings_pearson-true_ratings).sum())
print("Error with cosine similarity:",np.absolute(pred_ratings_cosine-true_ratings).sum())

Error with Pearson similarity: 145.33615507214182
Error with cosine similarity: 143.74130914160054


At first sight, the two similarity metrics seem sensibly equivalent.

Actually, in 1998 John S. Breese and al. found empirically in <i>Empirical Analysis of Predictive Algorithms for Collaborative Filtering</i> [BHK98] that the best similarity metric for collaborative filtering was <b>Pearson correlation to the power 2.5</b>.

# <a id="sec3"></a> 3. Improving the model
<br><br>
The collaborative filtering approach suffers mainly from scarcity and scalability issues.
In order to improve the system's efficiency, a natural approach is dimensionality reduction. The idea is to map the information contained in the rating matrix into a smaller matrix, which would be easier to process. Such a matrix can be obtained using Singular Value Decomposition (SVD). Following this analytical method, any matrix of size n x m can be decomposed as a product of three matrices of sizes <i>n x k</i>, <i>k x k</i>, and <i>k x m</i>.
<br><br>
<img src="img/singular_value_decomposition.png" width="500px">
<br><br>
This decomposition can be easily obtained with a dense matrix, however in our case the result matrices are not defined due to the missing values in our matrix.
Although methods exist to fill the matrix with placeholder values in order to obtain a dense matrix (for example by replacing missing values with the user's average rating), some information is lost in the process because missing ratings are part of users' behavior and also convey information.
<br><br>
Therefore, we choose to use a different method which can be applied to sparce matrices, exploiting the fact that ratings are positive values: Non-negative Matrix Factorization.
<br><br>
## <a id="sec3-1"></a> 3.1. Non-negative Matrix Factorization
<br>
Instead of factorizing our matrix R via SVD, we are trying to find two matrices P and Q directly with the goal that when P and Q are multiplied back together the output matrix R’ is the closest approximation of R and no longer a sparse matrix.
<br><br>
<img src="img/nmf.png" width="500px">
<i>Source: <a>https://towardsdatascience.com/paper-summary-matrix-factorization-techniques-for-recommender-systems-82d1a7ace74
</a></i>
<br><br>
For each component of the reconstructed matrix, we compute the error as follows:
$$e_{ij}^2 = (r_{ij} - \sum_k p_{ik}q_{kj})^2 + \frac{\beta}{2}\sum_k (||P||^2+||Q||^2)$$
where the second term is a regularization term added to avoid overfitting.

To update P and Q, we can use a simple gradient descent algorithm. The gradient is approximated in the first order and the components are updated as follows:

$$p_{ik}' = p_{ik} + \alpha\frac{\partial}{\partial p_{ik}}e_{ij}^2 = p_{ik} + \alpha(2e_{ij}q_{kj} - \beta p_{ik})$$
$$q_{kj}' = q_{kj} + \alpha\frac{\partial}{\partial q_{kj}}e_{ij}^2 = q_{kj} + \alpha(2e_{ij}p_{ik} - \beta q_{kj})$$

From there, we can build a very simple model which can be used to factorize our matrix.

In [18]:
# Reusable tool code

def matrix_factorization(R, P, Q, K, steps=5, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        print("Epoch",step)
        for i in range(len(R)):
            if i%100==0:
                print(round((i/len(R))*50,2),"%",end="\r")
            for j in range(len(R[i])):
                if R[i][j] != np.nan:
                    if R[i][j] > 0:
                        eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                        for k in range(K):
                            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                            Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        #eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            if i%100==0:
                print(round(((i/len(R))*50)+50,2),"%",end="\r")
            for j in range(len(R[i])):
                if R[i][j] != np.nan:
                    if R[i][j] > 0:
                        e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                        for k in range(K):
                            e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        print("Error:",e)
        if e < 0.001:
            break
    return P, Q.T

We can now train the model. This step can take around five minutes.

In [19]:
R = rating_matrix
N = len(R)
M = len(R[0])
K = 2

P = np.random.rand(N,K)
Q = np.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)
print("Done")

Epoch 0
Error: 5774759.930196543
Epoch 1
Error: 3207828.3443274414
Epoch 2
Error: 2295127.4652669285
Epoch 3
Error: 1846720.89382152
Epoch 4
Error: 1588530.68833736
Done


## <a id="sec3-2"></a> 3.2. A working recommender system

<br>
The matrices computed can now be used to produce recommendations for users. We simply have to select the best predicted ratings among movies which were not already rated by the user. These will constitute our recommendations.

<div class="alert alert-warning">

**Exercice:**<br>
Implement a function that produces a list of k films' titles recommended to the user using the previously computed matrices.

</div>

In [21]:
# %load solutions/recommend.py
# Uncomment the line above to load a correction to this exercise.

# Reusable tool code

def recommend(user,P,Q,k=10):
    rated_movies = np.where(~np.isnan(rating_matrix[user,:]))[0]
    pred_ratings = [[m,np.dot(nP[user,:],nQ.T[:,m])] for m in range(Q.shape[0]) if m not in rated_movies]
    top_ratings = sorted(pred_ratings, key = lambda x: x[1], reverse=True)[:k]
    
    return [df_movies[df_movies["movie_id"] == m[0]]["title"].values[0] for m in top_ratings]

<div class="alert alert-warning">
Try your function using the cell below and see what movies your recommender system pulls out.
</div>

In [22]:
test_user = 33

recommend(test_user,nP,nQ)

['Godfather, The (1972)',
 "Schindler's List (1993)",
 'Star Wars: Episode IV - A New Hope (1977)',
 'Raiders of the Lost Ark (1981)',
 'Casablanca (1942)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Usual Suspects, The (1995)',
 'Sixth Sense, The (1999)',
 'Silence of the Lambs, The (1991)',
 'Saving Private Ryan (1998)']

As we can see, the model above is far more efficient than the initial one in terms of execution time. It is able to make fast (less than one second) recommendations and thus can be used online, where users do not generally accept to wait more than a few seconds to get a result.
<br><br>
Obviously, the matrix factorization affects the precision of the model but it is nevertheless justified by the major increase in terms of recommendation speed.
<br><br>
From there, in order to increase the recommendation power of this model, the matrix factorization algorithm could be trained longer, with a higher factorization order and parameters could be tuned so the output better encapsulates the information from the ratings matrix. We could also consider using some additional information on users and movies in order to compute clusters which would help make the recommender system even more performant.

## Conclusion

Collaborative filtering offers an efficient recommendation capacity, and presents the major advantage to require no external information on users or items apart from previous ratings.
However it shows a few limitations:
- The latent features mapped by the model cannot be easily interpreted, as could be the genre of a movie for instance.
- It suffers from a "cold start": when a new user appears, the system is not able to produce good recommendations until a minimum number of ratings have been made.

In practice, a good approach would be to complete a collaborative filtering model with features extracted from user and items information, thus using a more content-based approach. Overall, the main objective remains to find a good tradeoff between computational complexity and precision of recommendations.

<u>References & ressources</u>

- Movielens dataset (1M) available at: <a>https://grouplens.org/datasets/movielens/</a>
- https://www.lri.fr/~antoine/Courses/Master-ISI/section-app-collaboratif.pdf
- Empirical Analysis of Predictive Algorithms for Collaborative Filtering [BHK98] (https://arxiv.org/pdf/1301.7363.pdf)
- http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf
- https://towardsdatascience.com/paper-summary-matrix-factorization-techniques-for-recommender-systems-82d1a7ace74
- http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/#source-code
- B. Dahlen, J. Konstan, J. Herlocker, N. Good, A. Borchers, and J. Riedl, “Jump-starting MovieLens: User benefits of starting a collaborative filtering system with ‘dead data,” Technical Report 98-017, University of Minnesota, Minneapolis, MN, March, 1998.
- http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/#source-code